К сожалению методами ИИ задачу решить пока не получилось. Перевод сделан традиционными алгоритмами.

In [5]:
import pandas as pd

import requests
import re
import string

import pymorphy2
lemmer = pymorphy2.MorphAnalyzer()

In [6]:
df = pd.read_csv('../lr3/witcher_words.csv')

In [131]:
def translate_russian_to_elvish(text: str) -> str:
    """
    Переводит русский текст на язык Старшей Речи.
    Слова, перевод которым подобрать не получилось, латинизируются.
    Присутствует обработка некоторых устойчивых выражений из словаря.
    """

    # Унифицируем е/ё:
    text = text.replace('ё', 'е')

    # Результирующие подстроки для склеивания в конце.
    result_subs = []

    # Отделяем слова от пунктуационных знаков:
    subs = re.split(r'(?<=\.|!|\?|,|;|:|\s)|(?=\.|!|\?|,|;|:|\s)', text)

    next_i = 0
    for i, sub in enumerate(subs):
        if i < next_i:
            continue

        next_i += 1
        if sub in string.punctuation + ' ':
            result_subs.append(sub)
            continue

        word = sub
        is_title = word.replace('\'', '').istitle()
        word = word.lower()
        word = _word_to_lemm(word)

        # Текущее слово может быть началом какого-то устойчивого выражения, проверим это.
        it_is_real_phrase = False
        phrases = _find_phrases_that_starts_with_russian_word(word)
        if phrases and i != len(subs) - 1:
            # Будем прибавлять сюда последующие пробелы и слова, пока не встретим что-то лишнее либо не убедимся
            # в том, что это выражение - действительная устойчивая фраза из словаря.
            summ_word = word

            # Количество итераций для пропуска во внешнем цикле (через `next_i`)
            count_to_pass = 0

            for sub in subs[i + 1:]:
                if sub in '.!?':
                    break

                count_to_pass += 1
                if sub not in string.punctuation + ' ':
                    sub = _word_to_lemm(sub.lower())
                elif sub == ',':
                    # Опускаем запятые между словами потенциальной фразы.
                    sub = ''

                summ_word += sub

                need_continue = False
                if summ_word in phrases:
                    it_is_real_phrase = True
                    break

                for phrase in phrases:
                    if summ_word in phrase:
                        need_continue = True
                        break

                if not need_continue:
                    break

            if it_is_real_phrase:
                word = summ_word
                next_i += count_to_pass
                translation = _try_to_find_russian_word_translation(word)

        if not it_is_real_phrase:
            # Стандартная обработка одиночного слова с подбором синонимов через сайт.
            translation = None
            try:
                translation = _try_to_find_russian_word_translation(word)
            except ValueError:
                for synonym in _synonyms_of_word(word):
                    try:
                        translation = _try_to_find_russian_word_translation(synonym)
                    except ValueError:
                        continue

            if translation is None:
                translation = _transliterate_russian_to_latin(word)

        if is_title:
            translation = translation.capitalize()

        result_subs.append(translation)

    return ''.join(result_subs)


def _word_to_lemm(word: str) -> str:
    """Переводит слово/словосочетание в начальную форму."""
    parts = []
    for part in word.split():
        part = lemmer.parse(part)[0].normal_form.replace('ё', 'е')
        parts.append(part)

    return ' '.join(parts)


def _try_to_find_russian_word_translation(word: str) -> str:
    """Ищет перевод русского слова/словосочетания в датасете."""
    result = df.loc[df['translation'] == word, 'text']
    if not result.empty:
        return result.values[0]
    else:
        raise ValueError


def _find_phrases_that_starts_with_russian_word(word: str) -> list[str]:
    return df.loc[df['translation'].str.startswith(word) & df['translation'].str.contains(' '), 'translation'].tolist()


def _synonyms_of_word(word: str) -> str:
    """Выдаёт всевозможные синонимы заданного слова. Использует сетевой запрос."""
    html = requests.get(f'https://text.ru/synonym/{word}').text
    
    match = re.search(r'<meta name=\"description\" content=\"Синонимы к слову [^—]*:([^\"]+)\" />', html)
    if match is None:
        return []

    text = match.group(1)
    synonyms = [synonym.strip() for synonym in text.strip().split('—')]
    synonyms.remove('')

    return synonyms


def _transliterate_russian_to_latin(russian_word: str) -> str:
    """Функция для латинизации русского слова."""
    translit_dict = {
        'а': 'a', 
        'б': 'b', 
        'в': 'v', 
        'г': 'g', 
        'д': 'd', 
        'е': 'e', 
        'ё': 'yo',
        'ж': 'zh', 
        'з': 'z', 
        'и': 'i', 
        'й': 'y', 
        'к': 'k', 
        'л': 'l', 
        'м': 'm',
        'н': 'n', 
        'о': 'o', 
        'п': 'p', 
        'р': 'r', 
        'с': 's', 
        'т': 't', 
        'у': 'u',
        'ф': 'f', 
        'х': 'kh', 
        'ц': 'ts', 
        'ч': 'ch', 
        'ш': 'sh', 
        'щ': 'shch', 
        'ъ': '',
        'ы': 'y', 
        'ь': '', 
        'э': 'e', 
        'ю': 'yu', 
        'я': 'ya',
    }

    latin_word = ''.join(translit_dict.get(symbol, symbol) for symbol in russian_word.lower())
    return latin_word

In [109]:
translate_russian_to_elvish('Здравствуй, Геральт Белый Волк! Моё имя - Аваллак\'х! Помоги мне найти ласточку!')

"Cead, Geralt Gwynbleidd! Me aymm - Avallak'kha! Pomoch aé nayti zireael!"

In [110]:
translate_russian_to_elvish('Прощай, дочь. Мы еще встретимся.')

'Va faill, luned. Sinn aris vstretitsya.'

In [111]:
translate_russian_to_elvish('Когда обед? Я хочу есть.')

'Cathain obed? Aé wett ithte.'

In [112]:
translate_russian_to_elvish('Почему этот курс такой сложный? Я не знаю.')

'Que etot rider takoy slozhnyy? Aé neen ifit.'

In [113]:
translate_russian_to_elvish('Крепость Старого Моря прекрасна летом. Он здесь как картина.')

"Caer a'muirehen elaine eate. Ei anseo conas eie."

In [114]:
translate_russian_to_elvish('Привет, я в порядке, спасибо.')

'Cead, esseath ghnath, meas.'

In [115]:
translate_russian_to_elvish('Что ты хочешь?')

"Que te eras'wett?"

In [116]:
translate_russian_to_elvish('Здарова! Привет всем!')

'Zdarov! Ceadmil!'

In [117]:
translate_russian_to_elvish('Не забывай, Цири. Всегда, когда что-то кончается, что-то начинается.')

"Neen zabyvat, Tsiri. Evellienn, cathain va'esse deireadh aep eigean va'esse eighe faidh'ar."

In [124]:
translate_russian_to_elvish('Это в настоящее время нельзя делать.')

'А faoi lathair ayd tuve.'

In [132]:
translate_russian_to_elvish('Ты что! Следи за огнем! Костер почти прогорел.')

"Esseath que! Aereeh't do aenye! An wenn beagnach muchfar."